In [82]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Libraries for data preprocessing
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [83]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [84]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [85]:
df = normalize_text(df)

In [86]:
df.sample(5)

,sentiment,content
18401,worry,good night fellow twitterati back work tomorrow
23114,happiness,need yummy breakfast shift
5890,hate,waste way much paper office nothing worthy alm...
28476,relief,crumcake that s relief feel better knowing her...
31435,worry,need follower


In [89]:
a = df['sentiment'].isin(['sadness', 'happiness'])
df = df[a]

In [91]:
df['sentiment'] = df['sentiment'].replace({'happiness':1, 'sadness':0})

C:\Users\saeed\AppData\Local\Temp\ipykernel_51120\2609172714.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'happiness':1, 'sadness':0})


In [92]:
df

,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday
...,...,...
39986,1,going watch boy striped pj s hope cry
39987,1,gave bike thorough wash degrease grease it thi...
39988,1,amazing time last night mcfly incredible
39994,1,succesfully following tayla


In [93]:
df.value_counts()

sentiment  content                                                                       
1          happy mother s day                                                                8
           happy star war day                                                                8
0          headache                                                                          5
1          good morning everyone                                                             5
           happy mother day                                                                  5
                                                                                            ..
           yum mother s day lunch food business burnside delicious gorgeous day              1
           yum whole box cooky                                                               1
           yummy taco jack box                                                               1
           yummy yum cha mother s day mum dad arcel egg

In [94]:
vectorizer = CountVectorizer(max_features = 1000)
x = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [95]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state= 42)

In [97]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/adnansaeed11/19-SESSION.mlflow')
dagshub.init(repo_owner='adnansaeed11', repo_name='19-SESSION', mlflow=True)

mlflow.set_experiment('Logistic Regression Baseline Model')

Accessing as adnansaeed11

Initialized MLflow to track repo "adnansaeed11/19-SESSION"

Repository adnansaeed11/19-SESSION initialized!

2025/07/06 15:45:46 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline Model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/14a6697035714742a61674110f5ad26b', creation_time=1751798748630, experiment_id='1', last_update_time=1751798748630, lifecycle_stage='active', name='Logistic Regression Baseline Model', tags={}>

In [105]:
from joblib import dump
import os

with mlflow.start_run():

    # Log preprocessing params
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)

    # Train model
    model = LogisticRegression()
    model.fit(x_train, y_train)

    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")

    # Evaluate
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # Save and log model manually
    model_path = "model.joblib"
    dump(model, model_path)
    mlflow.log_artifact(model_path)

    # Save and log notebook
    notebook_path = "exp1.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)

    # Print results
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")


Accuracy: 0.776867469879518
Precision: 0.7690058479532164
Recall: 0.7773399014778325
F1 Score: 0.7731504164625184
🏃 View run beautiful-panda-606 at: https://dagshub.com/adnansaeed11/19-SESSION.mlflow/#/experiments/1/runs/d663c65ad4a14dcca2741c2c0b9a6a1f
🧪 View experiment at: https://dagshub.com/adnansaeed11/19-SESSION.mlflow/#/experiments/1
